In [26]:
import timeit
import numpy as np
from scipy.spatial.distance import cdist

# furthest point sampling

In [50]:
def furthest_point_sampling_a(points, k, seed=None):
    """
    Select k points from an array of points such that the distance between any two selected points
    is maximized.

    Args:
        points (np.array): array of shape (N, d) containing N points in d-dimensional space
        k (int): number of points to select

    Returns:
        np.array: array of shape (k, d) containing the selected points
    """

    if seed is not None:
        np.random.seed(seed)

    # Select the first point randomly
    selected = [np.random.randint(points.shape[0])]
    dists = np.zeros(points.shape[0])

    for i in range(1, k):
        # Compute the distance between each point and the nearest selected point
        for j, point in enumerate(points):
            dists[j] = np.min([np.linalg.norm(point - points[selected_point]) for selected_point in selected])

        # Select the point with the largest distance to the nearest selected point
        selected.append(np.argmax(dists))

    return points[selected]


In [54]:
def furthest_point_sampling_b(points, k, seed=None):
    """
    Select k points from an array of points such that the distance between any two selected points
    is maximized.

    Args:
        points (np.array): array of shape (N, d) containing N points in d-dimensional space
        k (int): number of points to select

    Returns:
        np.array: array of shape (k, d) containing the selected points
    """

    if seed is not None:
        np.random.seed(seed)
    
    # Select the first point randomly
    selected = [np.random.randint(points.shape[0])]
    dists = np.linalg.norm(points - points[selected[0]], axis=1)

    for i in range(1, k):
        # Select the point with the largest distance to the nearest selected point
        selected.append(np.argmax(dists))
        dists = np.minimum(dists, np.linalg.norm(points - points[selected[-1]], axis=1))

    return points[selected]


In [55]:
def furthest_point_sampling_c(points, k, seed=None):
    """
    Select k points from an array of points such that the distance between any two selected points
    is maximized.

    Args:
        points (np.array): array of shape (N, d) containing N points in d-dimensional space
        k (int): number of points to select

    Returns:
        np.array: array of shape (k, d) containing the selected points
    """

    if seed is not None:
        np.random.seed(seed)
    
    # Initialize a set to store the indices of the selected points
    selected = [np.random.randint(points.shape[0])]
    
    # Iteratively select the next k-1 points
    for _ in range(1, k):
        dists = cdist(points, points[selected], metric='euclidean').min(axis=1)

        # Select the point with the largest distance to the nearest selected point
        selected.append(np.argmax(dists))
    
    return points[selected]    


# checking equivalence of approaches

In [56]:
points = np.random.rand(100, 2)

selected_points_a = furthest_point_sampling_a(points, 10, seed=0)
selected_points_b = furthest_point_sampling_b(points, 10, seed=0)
selected_points_c = furthest_point_sampling_c(points, 10, seed=0)

(selected_points_a == selected_points_b).all() and (selected_points_b == selected_points_c).all()

True

# benchmarking

## toy case

In [57]:
NUM_RUNS = 100

print('furthest_point_sampling_a:', 
      timeit.Timer(lambda: furthest_point_sampling_a(np.random.rand(100, 2), 10)).timeit(number=NUM_RUNS) / NUM_RUNS)
print('furthest_point_sampling_b:', 
      timeit.Timer(lambda: furthest_point_sampling_b(np.random.rand(100, 2), 10)).timeit(number=NUM_RUNS) / NUM_RUNS)
print('furthest_point_sampling_c:', 
      timeit.Timer(lambda: furthest_point_sampling_c(np.random.rand(100, 2), 10)).timeit(number=NUM_RUNS) / NUM_RUNS)


furthest_point_sampling_a: 0.010270723749999888
furthest_point_sampling_b: 6.807707999996637e-05
furthest_point_sampling_c: 7.164332999764156e-05


## real-world case

In [58]:
NUM_RUNS = 10

print('furthest_point_sampling_b:', 
      timeit.Timer(lambda: furthest_point_sampling_b(np.random.rand(20000, 512), 100)).timeit(number=NUM_RUNS) / NUM_RUNS)

furthest_point_sampling_b: 2.0521284500000094
